Scrape the Wikipedia page, using Beautiful Soup package

Get url, decode page source

In [1]:
import requests
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

Find source code containing table, extract table columns

In [3]:
My_table=soup.find('table',{'class':'wikitable sortable'})

In [4]:
rows=My_table.find_all('tr')
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')]

Extract table values: 
1.Ignore rows with borough 'not assigned'
2.Set not assigned neighbourhood with the same value of borough
3.Sort the table by Postal code for convenience

In [5]:
import pandas as pd
df=pd.DataFrame(columns=columns)
for i in range(1,len(rows)):
    tds=rows[i].find_all('td')
    values=[v.text.replace('\n','') for v in tds]
    if values[1].lower()!='not assigned':
        if values[2].lower()=='not assigned':
            values[2]=values[1]
        df=df.append(pd.Series(values,index=columns),ignore_index=True)
df.sort_values(by='Postcode',inplace=True)
df.reset_index(drop=True,inplace=True)
#print(df)

Merge rows with same postal code, print the final table

In [26]:
df2=pd.DataFrame(columns=columns)
k=-1
i=0
while i+1<len(df):
    df2=df2.append(df.iloc[i])
    k+=1
    j=i+1
    while df.iloc[i]['Postcode']==df.iloc[j]['Postcode']:
        df2.iloc[k]['Neighbourhood']=df2.iloc[k]['Neighbourhood']+', '+df.iloc[j]['Neighbourhood']
        j+=1
    i=j    
df2.reset_index(drop=True,inplace=True)
#print(df2)    

Print the shape of final table

In [27]:
print(df2.shape)

(102, 3)


Import csv file

In [28]:
post2coor=pd.read_csv("Geospatial_Coordinates.csv")

Look up coordinates and add new columns to the final table df2

In [29]:
coorla=[]
coorlo=[]
for i in range(len(df2)):
    coor=post2coor.loc[post2coor['Postal Code']==df2.iloc[i]['Postcode']]
    coorla.append(coor.iloc[0]['Latitude'])
    coorlo.append(coor.iloc[0]['Longitude'])
df2['Latitude']=coorla
df2['Longitude']=coorlo
pd.options.display.max_colwidth = 20
print(df2)

    Postcode      Borough        Neighbourhood   Latitude  Longitude
0        M1B  Scarborough       Rouge, Malvern  43.806686 -79.194353
1        M1C  Scarborough  Port Union, Roug...  43.784535 -79.160497
2        M1E  Scarborough  Guildwood, Morni...  43.763573 -79.188711
3        M1G  Scarborough               Woburn  43.770992 -79.216917
4        M1H  Scarborough            Cedarbrae  43.773136 -79.239476
..       ...          ...                  ...        ...        ...
97       M9M   North York     Humberlea, Emery  43.724766 -79.532242
98       M9N         York               Weston  43.706876 -79.518188
99       M9P    Etobicoke            Westmount  43.696319 -79.532242
100      M9R    Etobicoke  Richview Gardens...  43.688905 -79.554724
101      M9V    Etobicoke  Albion Gardens, ...  43.739416 -79.588437

[102 rows x 5 columns]
